# Add/Remove Machine to Device Groups based on Machine Tag

    Author: Jing Nghik
    Last Updated 6/23/2020

Purpose: This notebook is used to add device's to a certain group based on the machine tag.

## Requirements: 

[Machine groups already pre-configured by Device tag](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/machine-groups)

    You need to have the machine groups already configured that filter based on the device tag element.

[Proper MDATP API Token](https://docs.microsoft.com/en-us/windows/security/threat-protection/microsoft-defender-atp/exposed-apis-create-app-webapp)
        
    Microsoft Defender ATP exposes much of its data and actions through a set of programmatic APIs. Those APIs will help you automate work flows and innovate based on Microsoft Defender ATP capabilities.


## Replace variables with pertaining to your environment

In [40]:
############ This is the credentials used to authenticate with the Graph API ###################
$credentials = @{
    tenantDomain = "domain.com"
    tenantId = "xxxxxxx-xxxx-xxxx-xxxx-xxxxxxxx"        ## AAD Tenant ID
    client_id = "ae4fa870-23af-4ae5-a6ed-xxxxx"     ## Application Client ID
    client_secret = "x~xxxxxxx~xxxxx~xxxxxxxxx-xxx"   ## Application Client Secret
}

## Import required modules and credentials

In [11]:
$modules = @('pp-core','graph') ## These are the modules to import 
$credentialPath = "~/.credentials/graph.credential" ## Where to store credentials
################## Import Graph Authentication Module #############################
ForEach ($module in $modules) {
    Import-Module "~/Notebooks/Powershell-Playground/PowerShell/custom-modules/$module" -Force  ## import module 
    Get-Command -Module "$module"                                                        ## list commands in this module 
}

$credentials = New-StoreCredentials -credentialPath $credentialPath -credentialArray $credentials


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Get-DecryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-EncryptedStringUsingMasterPassword             0.0        pp-core
Function        Get-HashOfString                                   0.0        pp-core
Function        Get-MasterPassword                                 0.0        pp-core
Function        Get-StoredCredentials                              0.0        pp-core
Function        Invoke-CheckCredentials                            0.0        pp-core
Function        New-StoreCredentials                               0.0        pp-core
Function        Get-MSGraphAuthToken                               0.0        graph
Function        Invoke-GraphAuthentication                         0.0        graph
Function        Invoke-MSGraphQuery                        

## Authenticate with Graph API and get access Token for MDATP Resource

In [10]:
## Get a token if authenticates properly.
$authParams = @{
    tenantId = $credentials.tenantId
    client_id = $credentials.client_id
    client_secret = $credentials.client_secret
    resource = "https://api.securitycenter.windows.com"    ## resource Dont change since we want to query MDATP REST API Resource
    grant_type = "client_credentials"  ## This is using a appliation ID and secret to authenticate
}; $authParams

$tokenResponse = Invoke-GraphAuthentication -authParams $authParams
$tokenResponse


Name                           Value
----                           -----
grant_type                     client_credentials
client_id                      ae4fa870-23af-4ae5-a6ed-5ab1811858c7
resource                       https://api.securitycenter.windows.com
client_secret                  5~Q5En6ecx9d~ZtuqPf~YRBLw5S858-7WS
tenantId                       de40cf7e-ad5f-4245-a317-14be39cbb0ef

Authentication Parameters detected
Personal Access Token (PAT) grant_type

----------------------------------------------------------------------------
Authentiating with Microsoft Graph API using a Personal Access Token (PAT)
https://docs.microsoft.com/en-us/azure/storage/common/storage-auth-aad-app
----------------------------------------------------------------------------
Requesting Token at https://login.microsoftonline.com/de40cf7e-ad5f-4245-a317-14be39cbb0ef/oauth2/token


Received Token!
Connected and Access Token received and will expire 

token_type     : Bearer
expires_in     : 3599
e

In [6]:
## This is the authentication header for REST API Call
$authHeader = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($tokenResponse.access_token)" 
}

## The following commands will query the graph API and output the first 5 machines in table format
$URI = "https://api-us.securitycenter.windows.com/api/machines"
$Response = (Invoke-RestMethod -Method GET -Uri $URI -Headers $authHeader -body $body -ErrorAction Stop)
$Response.value | Select-Object -First 5 | Select-Object @('id','computerDnsName','machineTags') | FT


id                                       computerDnsName   machineTags
--                                       ---------------   -----------
fcb604d2f2b895cdbb296ee64040a4693ef230ab bdlabs-cent7-1    {}
bffd249f248a1d1b95e5f90922c9a1df776de35f jing-lab          {}
d1447e0ab03ed0f39c9e2e0a3ac3cc36cc8ecf78 jingtoso-desktop1 {group1}
a8fe7113c8bab16e43a818f0a972515da82e94f8 win10-oobe-test   {group1, group2}
4bfbac0c45e72411f445eed810f4a423be2a335d jingtoso-pc1      {group2}



## Identify machine, tag, and whether to add or remove the tag on the machine

In [7]:
$machineId = "4bfbac0c45e72411f445eed810f4a423be2a335d"  ## Machine ID to add or remove device group
$machineTag = "group1"
$action = "Add" ## This can be add or remove

## Add/Remove machine based on above variables

In [8]:
####### Creates the json body ####################
$jsonBody = @{
    Value = $machineTag
    Action = $action
}
$jsonBody = ($jsonbody | ConvertTo-JSON); $jsonBody

####### Calls MDATP Rest API and either add or remove machine tag #################
$URI = "https://api-us.securitycenter.windows.com/api/machines/$machineId/tags"; $URI
$Response = Invoke-RestMethod -Method POST -Uri $URI -Headers $authHeader -body $jsonBody -ErrorAction Stop

{
  "Value": "group1",
  "Action": "Add"
}
https://api-us.securitycenter.windows.com/api/machines/4bfbac0c45e72411f445eed810f4a423be2a335d/tags


## Get Machine list again to verify

In [9]:
$URI = "https://api-us.securitycenter.windows.com/api/machines"
$Response = (Invoke-RestMethod -Method GET -Uri $URI -Headers $authHeader -body $body -ErrorAction Stop)
$Response.value | Select-Object @('id','computerDnsName','machineTags') | FT


id                                       computerDnsName   machineTags
--                                       ---------------   -----------
fcb604d2f2b895cdbb296ee64040a4693ef230ab bdlabs-cent7-1    {}
bffd249f248a1d1b95e5f90922c9a1df776de35f jing-lab          {}
d1447e0ab03ed0f39c9e2e0a3ac3cc36cc8ecf78 jingtoso-desktop1 {group1}
a8fe7113c8bab16e43a818f0a972515da82e94f8 win10-oobe-test   {group1, group2}
4bfbac0c45e72411f445eed810f4a423be2a335d jingtoso-pc1      {group1, group2}
35c817f459246946bb02e9832e46c8e3260fe9dd jings-desktop     {group2}

